In [1]:
%matplotlib widget
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from physoce.tseries import rot

## Load datasets

### Surface buoy/frame - ADV and ADCP

In [2]:
nc_adcp_surf = 'nc_files/adcp_surface_orshelf.nc'
ds_adcp_surf = xr.open_dataset(nc_adcp_surf)

In [3]:
nc_adv_surf = 'nc_files/deployment0009_CE02SHSM-SBD11-04-VELPTA000-recovered_inst-velpt_ab_instrument_recovered_20190420T000000-20191022T153000.nc'
ds_adv_surf = xr.open_dataset(nc_adv_surf)
ds_adv_surf = ds_adv_surf.swap_dims({'obs':'time'})

### Benthic experiment package - ADV and ADCP

In [4]:
nc_adcp_benthic_list = sorted(glob('nc_files/benthic_adcp_orshelf/*avg.nc'))

In [5]:
nc_adv_benthic_list = sorted(glob('nc_files/benthic_adv_orshelf/*avg.nc'))

In [6]:
ds_adcp_benthic = xr.open_mfdataset(nc_adcp_benthic_list)
ds_adv_benthic = xr.open_mfdataset(nc_adv_benthic_list)

In [7]:
ds_adcp_benthic_deployment1 = xr.open_dataset(nc_adcp_benthic_list[0])

### Vertical coordinates

In [8]:
def add_adcp_vertical_coordinates(ds,inst_location):
    '''
    This function creates additional variables for keeping track of depths in the ADCP data:
    
    ** Bottom mounted **
    The dataset provided by OOI includes the following variables:

    'depth_from_pressure' is the total water column depth, calculated from the pressure sensor. Dimensions: (obs)
    'bin_depths' is the depth of each ADCP bin below the sea surface, which moves up and down. The bin depths constantly change in time. Dimensions: (obs, bin)

    'bin_height' is the height of each ADCP bin above the bottom. These heights are fixed, since the ADCP is mounted at the bottom. Dimensions: (bin)
    'bin_depths_avg' is the average depth below the surface for each ADCP bin. These depths are fixed in time since they are defined in relation to a time average. Dimensions: (bin)
    'sea_surface' is the height of the sea surface, relative to the time average. Dimensions: (obs)
    'bottom_depth_avg' is the average bottom depth, calculated from the pressure sensor. This is a single value with no dimensions. Dimensions: ()    
    '''
    
    if inst_location == 'bottom':
        ds['bin_height'] =  ds['depth_from_pressure'][0] - ds['bin_depths'][0,:]
        ds['bin_height'].attrs['units'] = 'm'

        ds['bin_depths_avg'] = np.mean(ds['depth_from_pressure'])-ds['bin_height']
        ds['bin_depths_avg'].attrs['units'] = 'm'

        ds['sea_surface'] = ds['depth_from_pressure']-np.mean(ds['depth_from_pressure'])
        ds['sea_surface'].attrs['units'] = 'm'

        ds['bottom_depth_avg'] = np.mean(ds['depth_from_pressure'])
        ds['bottom_depth_avg'].attrs['units'] = 'm'
        
    return ds

In [9]:
ds_adcp_benthic = add_adcp_vertical_coordinates(ds_adcp_benthic,inst_location='bottom')

#### Surface point measurement

Create depth variable for surface ADV
* Attributes for `pressure_mbar` variable state that surface pressure has been removed and that the units are 0.001 dbar (although this is not actually millibars)
* No info on blanking distance but 0.2 m is the minimum for a Nortek Aquadopp

Add `bin` dimension to allow combination with other datasets (even though there is only one bin)

Also create variable names for velocity that are consistent with other data sets

In [10]:
ds_adv_surf['bin_depths'] = ds_adv_surf['pressure_mbar']/1000 + 0.2 
ds_adv_surf['eastward_seawater_velocity'] = ds_adv_surf['eastward_velocity']
ds_adv_surf['northward_seawater_velocity'] = ds_adv_surf['northward_velocity']

In [11]:
ds_adv_surf = ds_adv_surf.expand_dims(bin=[0],axis=1)

In [12]:
ds_adv_surf['bin_depths_avg'] = ds_adv_surf['bin_depths'].mean(dim='time')
ds_adv_surf['bin_depths_avg'].attrs['units'] = 'm'

#### Surface ADCP

Create `bin_depths_avg` variable

In [13]:
ds_adcp_surf['bin_depths_avg'] = ds_adcp_surf['bin_depths'].mean(dim='time')
ds_adcp_surf['bin_depths_avg'].attrs['units'] = 'm'

#### Benthic ADV
* Note shift in pressure between deployments - not close to the depth of the ADCP in either deployment
* Also more limited in time availability
* Not including in combined dataset for now

In [14]:
ds_adv_benthic['eastward_seawater_velocity'] = ds_adv_benthic['vel3d_c_eastward_turbulent_velocity']/10
ds_adv_benthic['northward_seawater_velocity'] = ds_adv_benthic['vel3d_c_northward_turbulent_velocity']/10

In [15]:
plt.figure()
plt.plot(ds_adv_benthic['time'],ds_adv_benthic['seawater_pressure_mbar']/1000)
plt.plot(ds_adcp_benthic['time'],ds_adcp_benthic['depth_from_pressure'])
plt.legend(['ADV','ADCP'])
plt.ylabel('pressure/depth')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'pressure/depth')

### Data availability

In [16]:
plt.figure(figsize=(8,5))
plt.plot(ds_adv_surf['time'],ds_adv_surf['northward_velocity'].squeeze())
plt.plot(ds_adcp_surf['time'],ds_adcp_surf['northward_seawater_velocity'][:,0])
plt.plot(ds_adcp_benthic['time'],ds_adcp_benthic['northward_seawater_velocity'][:,0])
plt.plot(ds_adv_benthic['time'],ds_adv_benthic['vel3d_c_northward_turbulent_velocity']/10)
plt.legend(['surface point','surface ADCP','benthic ADCP','benthic point'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Align data in time

In [17]:
(ds_adv_surf_aligned,
 ds_adcp_surf_aligned,
 ds_adcp_benthic_aligned) = xr.align(ds_adv_surf,
                                    ds_adcp_surf,
                                    ds_adcp_benthic,
                                   join='inner',
                                   exclude='bin')

In [18]:
plt.figure(figsize=(8,5))
plt.plot(ds_adv_surf_aligned['time'],ds_adv_surf_aligned['northward_velocity'])
plt.plot(ds_adcp_surf_aligned['time'],ds_adcp_surf_aligned['northward_seawater_velocity'][:,0])
plt.plot(ds_adcp_benthic_aligned['time'],ds_adcp_benthic_aligned['northward_seawater_velocity'][:,0])
plt.legend(['surface point','surface ACP','benthic ADCP'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(ds_adcp_benthic_aligned['northward_seawater_velocity'].mean(dim='time'),
         ds_adcp_benthic_aligned['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['northward_seawater_velocity'].mean(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['northward_velocity'].mean(dim='time'),2,'o')
plt.gca().invert_yaxis()
plt.ylabel('depth [m]')
plt.xlabel('mean v [m/s]')
plt.legend(['bethic ADCP','surface ADCP','surface point'])
plt.subplot(1,2,2)
plt.plot(ds_adcp_benthic_aligned['northward_seawater_velocity'].std(dim='time'),
         ds_adcp_benthic_aligned['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['northward_seawater_velocity'].std(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['northward_velocity'].std(dim='time'),
         2,'o')
plt.gca().invert_yaxis()
plt.xlabel('std v [m/s]')
plt.suptitle('northward velocity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'northward velocity')

In [20]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(ds_adcp_benthic_aligned['eastward_seawater_velocity'].mean(dim='time'),
         ds_adcp_benthic_aligned['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['eastward_seawater_velocity'].mean(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['eastward_velocity'].mean(dim='time'),2,'o')
#plt.plot(ds_adv_benthic_aligned['vel3d_c_northward_turbulent_velocity'].mean(dim='time')/10,80,'s')
plt.gca().invert_yaxis()
plt.xlim([-0.25,0.25])
plt.ylabel('depth [m]')
plt.xlabel('mean u [m/s]')
plt.subplot(1,2,2)
plt.plot(ds_adcp_benthic_aligned['eastward_seawater_velocity'].std(dim='time'),
         ds_adcp_benthic_aligned['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['eastward_seawater_velocity'].std(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['eastward_velocity'].std(dim='time'),
         2,'o')
#plt.plot(ds_adv_benthic_aligned['vel3d_c_northward_turbulent_velocity'].std(dim='time')/10,80,'s')
plt.gca().invert_yaxis()
plt.xlim([0,1])
plt.xlabel('std u [m/s]')
plt.suptitle('eastward velocity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'eastward velocity')

In [21]:
nbins = np.shape(ds_adcp_surf['bin_depths'])[1]
rs = np.nan*np.zeros(nbins)
for bis in range(nbins):
    bib = np.argmin(np.abs(ds_adcp_benthic['bin_depths'].mean(dim='time').values-
                           ds_adcp_surf['bin_depths'].mean(dim='time').values[bis]))
    r = xr.corr(ds_adcp_surf_aligned['northward_seawater_velocity'][:,bis],
        ds_adcp_benthic_aligned['northward_seawater_velocity'][:,bib],
        dim='time')
    rs[bis] = r.values

In [22]:
plt.figure()
plt.plot(rs,ds_adcp_surf['bin_depths'].mean(dim='time'),'-^')
plt.gca().invert_yaxis()
plt.title('correlation between surface and bottom ADCPs')
plt.xlabel('r')
plt.ylabel('depth [m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'depth [m]')

Find depth of maximum correlation

In [23]:
maxri = np.argmax(rs)
depth_maxcorr = ds_adcp_surf['bin_depths'].max(dim='time').values[maxri]

In [24]:
zsurfi, = np.where(ds_adcp_surf_aligned['bin_depths'].max(dim='time') <= depth_maxcorr)
zboti, = np.where(ds_adcp_benthic_aligned['bin_depths'].min(dim='time') > depth_maxcorr)

## Investigate differences between surface and bottom ADCP

In [25]:
ds_adcp_surf_aligned['direction'] = np.arctan2(ds_adcp_surf_aligned['northward_seawater_velocity'],
                                              ds_adcp_surf_aligned['eastward_seawater_velocity'])
ds_adcp_surf_aligned['speed'] = np.sqrt(ds_adcp_surf_aligned['northward_seawater_velocity']**2 + 
                                              ds_adcp_surf_aligned['eastward_seawater_velocity']**2)

In [26]:
ds_adcp_benthic_aligned['direction'] = np.arctan2(ds_adcp_benthic_aligned['northward_seawater_velocity'],
                                              ds_adcp_benthic_aligned['eastward_seawater_velocity'])

Look only at times where speed is sufficiently large to determine an angle

In [27]:
plt.figure()
si, = np.where(ds_adcp_surf_aligned['speed'][:,zsurfi[-1]] > 0.1)

plt.plot(ds_adcp_surf_aligned['time'][si],(ds_adcp_surf_aligned['direction'][si,zsurfi[-1]]
          -ds_adcp_benthic_aligned['direction'][si,zboti[-1]])*180/np.pi,'.')
plt.title('angle difference at '+str(depth_maxcorr)[:4]+'m')
plt.ylabel('[deg]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '[deg]')

In [28]:
tis1, = np.where((ds_adcp_benthic_aligned['time'] <= ds_adcp_benthic_deployment1['time'][-1]) & 
               (ds_adcp_surf_aligned['speed'][:,zsurfi[-1]] > 0.1)) 

tis2, = np.where((ds_adcp_benthic_aligned['time'] > ds_adcp_benthic_deployment1['time'][-1]) & 
               (ds_adcp_surf_aligned['speed'][:,zsurfi[-1]] > 0.1)) 

plt.figure()
plt.hist((ds_adcp_surf_aligned['direction'][tis1,zsurfi[-1]]
          -ds_adcp_benthic_aligned['direction'][tis1,zboti[-1]])*180/np.pi,
         bins=180,range=[-360,360])

plt.hist((ds_adcp_surf_aligned['direction'][tis2,zsurfi[-1]]
          -ds_adcp_benthic_aligned['direction'][tis2,zboti[-1]])*180/np.pi,
         bins=180,range=[-360,360]);
plt.xlim([-360,360]);
plt.legend(['deployment 5','deployment 6'])
plt.xlabel('angle difference (surface - benthic) [deg]');
plt.ylabel('count');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Look only at directions between -180 to 180 deg and take average

In [29]:
dir_diff_deg = (ds_adcp_surf_aligned['direction'][:,zsurfi[-1]]
                                             -ds_adcp_benthic_aligned['direction'][:,zboti[-1]])*180/np.pi

tis1sub, = np.where((ds_adcp_benthic_aligned['time'] <= ds_adcp_benthic_deployment1['time'][-1]) & 
                    (ds_adcp_surf_aligned['speed'][:,zsurfi[-1]] > 0.1) & 
                    (dir_diff_deg > -180) & (dir_diff_deg < 180))

tis2sub, = np.where((ds_adcp_benthic_aligned['time'] > ds_adcp_benthic_deployment1['time'][-1]) & 
                    (ds_adcp_surf_aligned['speed'][:,zsurfi[-1]] > 0.1) & 
                    (dir_diff_deg > -180) & (dir_diff_deg < 180))

In [30]:
plt.figure()
plt.hist(dir_diff_deg[tis1sub],bins=180,range=[-360,360]);
plt.xlim([-180,180]);
plt.hist(dir_diff_deg[tis2sub],bins=180,range=[-360,360]);
plt.xlim([-180,180]);
yl = plt.ylim()
plt.legend(['deployment 5','deployment 6'])

dir_diff_mean1 = np.mean(dir_diff_deg[tis1sub])
dir_diff_mean2 = np.mean(dir_diff_deg[tis2sub])

plt.plot([dir_diff_mean1,dir_diff_mean1],yl)
plt.plot([dir_diff_mean2,dir_diff_mean2],yl)
plt.text(dir_diff_mean2,yl[1],str(dir_diff_mean2.values)[:5],verticalalignment='top')
plt.text(dir_diff_mean1,yl[1],str(dir_diff_mean1.values)[:5],verticalalignment='top')
plt.ylim(yl);
plt.xlabel('angle difference (surface - benthic) [deg]');
plt.ylabel('count');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Make compass corrections

Rotate benthic ADCP vectors using mean angles calculated above

In [31]:
ds_adcp_benthic_aligned_corrected = ds_adcp_benthic_aligned.copy().load()

uc1,vc1 = rot(ds_adcp_benthic_aligned['eastward_seawater_velocity'].isel(time=tis1),
           ds_adcp_benthic_aligned['northward_seawater_velocity'].isel(time=tis1),
           dir_diff_mean1)

uc2,vc2 = rot(ds_adcp_benthic_aligned['eastward_seawater_velocity'].isel(time=tis2),
           ds_adcp_benthic_aligned['northward_seawater_velocity'].isel(time=tis2),
           dir_diff_mean2)

ds_adcp_benthic_aligned_corrected['eastward_seawater_velocity'][tis1,:] = uc1
ds_adcp_benthic_aligned_corrected['northward_seawater_velocity'][tis1,:] = vc1

ds_adcp_benthic_aligned_corrected['eastward_seawater_velocity'][tis2,:] = uc2
ds_adcp_benthic_aligned_corrected['northward_seawater_velocity'][tis2,:] = vc2

ds_adcp_benthic_aligned_corrected['direction'] = np.arctan2(ds_adcp_benthic_aligned_corrected['northward_seawater_velocity'],
                                              ds_adcp_benthic_aligned_corrected['eastward_seawater_velocity'])

In [32]:
plt.figure()
si, = np.where(ds_adcp_surf_aligned['speed'][:,zsurfi[-1]] > 0.1)

plt.plot(ds_adcp_surf_aligned['time'][si],(ds_adcp_surf_aligned['direction'][si,zsurfi[-1]]
          -ds_adcp_benthic_aligned_corrected['direction'][si,zboti[-1]])*180/np.pi,'.')
plt.title('angle difference at '+str(depth_maxcorr)[:4]+'m')
plt.ylabel('[deg]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '[deg]')

In [33]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(ds_adcp_benthic_aligned_corrected['northward_seawater_velocity'].mean(dim='time'),
         ds_adcp_benthic_aligned_corrected['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['northward_seawater_velocity'].mean(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['northward_velocity'].mean(dim='time'),2,'o')
plt.gca().invert_yaxis()
plt.ylabel('depth [m]')
plt.xlabel('mean v [m/s]')
plt.legend(['bethic ADCP','surface ADCP','surface point'])
plt.subplot(1,2,2)
plt.plot(ds_adcp_benthic_aligned_corrected['northward_seawater_velocity'].std(dim='time'),
         ds_adcp_benthic_aligned_corrected['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['northward_seawater_velocity'].std(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['northward_velocity'].std(dim='time'),
         2,'o')
plt.gca().invert_yaxis()
plt.xlabel('std v [m/s]')
plt.suptitle('northward velocity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'northward velocity')

In [34]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(ds_adcp_benthic_aligned_corrected['eastward_seawater_velocity'].mean(dim='time'),
         ds_adcp_benthic_aligned_corrected['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['eastward_seawater_velocity'].mean(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['eastward_velocity'].mean(dim='time'),2,'o')
#plt.plot(ds_adv_benthic_aligned['vel3d_c_northward_turbulent_velocity'].mean(dim='time')/10,80,'s')
plt.gca().invert_yaxis()
plt.xlim([-0.25,0.25])
plt.ylabel('depth [m]')
plt.xlabel('mean u [m/s]')
plt.subplot(1,2,2)
plt.plot(ds_adcp_benthic_aligned['eastward_seawater_velocity'].std(dim='time'),
         ds_adcp_benthic_aligned['bin_depths'].mean(dim='time'),'^')
plt.plot(ds_adcp_surf_aligned['eastward_seawater_velocity'].std(dim='time'),
         ds_adcp_surf_aligned['bin_depths'].mean(dim='time'),'.')
plt.plot(ds_adv_surf_aligned['eastward_velocity'].std(dim='time'),
         2,'o')
#plt.plot(ds_adv_benthic_aligned['vel3d_c_northward_turbulent_velocity'].std(dim='time')/10,80,'s')
plt.gca().invert_yaxis()
plt.xlim([0,1])
plt.xlabel('std u [m/s]')
plt.suptitle('eastward velocity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'eastward velocity')

## Combine data sets

In [35]:
ds_adcp_surf_aligned_sub = ds_adcp_surf_aligned.isel(bin=zsurfi)

In [36]:
ds_adcp_benthic_aligned_sub = ds_adcp_benthic_aligned_corrected.isel(bin=zboti)

In [37]:
ds_adcp_surf_aligned_sub = ds_adcp_surf_aligned_sub.swap_dims({'bin':'bin_depths_avg'})
ds_adcp_surf_aligned_sub = ds_adcp_surf_aligned_sub.reset_coords('bin')

In [38]:
ds_adcp_surf_aligned_sub2 = ds_adcp_surf_aligned_sub[['eastward_seawater_velocity','northward_seawater_velocity',
                                                            'bin_depths']]

In [39]:
ds_adcp_benthic_aligned_sub = ds_adcp_benthic_aligned_sub.swap_dims({'bin':'bin_depths_avg'})
ds_adcp_benthic_aligned_sub = ds_adcp_benthic_aligned_sub.reset_coords('bin')

In [40]:
ds_adcp_benthic_aligned_sub2 = ds_adcp_benthic_aligned_sub[['eastward_seawater_velocity','northward_seawater_velocity',
                                                            'bin_depths',
                                                            'sea_surface','depth_from_pressure']]

In [41]:
ds_adv_surf_aligned_sub = ds_adv_surf_aligned.swap_dims({'bin':'bin_depths_avg'})

In [42]:
ds_adv_surf_aligned_sub = ds_adv_surf_aligned_sub.reset_coords('bin')

In [43]:
ds_adv_surf_aligned_sub2 = ds_adv_surf_aligned_sub[['eastward_seawater_velocity','northward_seawater_velocity',
                                                            'bin_depths']]

In [44]:
ds_merged = xr.concat([ds_adv_surf_aligned_sub2,
                      ds_adcp_surf_aligned_sub2,
                      ds_adcp_benthic_aligned_sub2],
                     dim='bin_depths_avg',coords='minimal',data_vars='minimal',compat='override')

In [45]:
ds_merged

<xarray.Dataset>
Dimensions:                      (bin_depths_avg: 37, time: 17510)
Coordinates:
  * time                         (time) datetime64[ns] 2019-04-20T00:30:00 .....
    obs                          (time) int32 2 3 4 5 ... 17813 17814 17815
  * bin_depths_avg               (bin_depths_avg) float64 1.894 9.148 ... 38.6
Data variables:
    depth_from_pressure          (time) float64 80.06 79.97 79.9 ... 81.07 81.13
    sea_surface                  (time) float64 -0.7068 -0.7922 ... 0.3683
    eastward_seawater_velocity   (time, bin_depths_avg) float64 -0.1552 ... 0...
    northward_seawater_velocity  (time, bin_depths_avg) float64 -0.4895 ... 0...
    bin_depths                   (time, bin_depths_avg) float64 1.842 ... 38.98
Attributes:
    node:                               SBD11
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             velpt_ab_instrument_recovered
    featureType:                        point
    creator_email:                      
    publisher_name:                     Ocean Observatories Initiative
    date_modified:                      2020-08-16T12:20:33.910748
    keywords:                           
    cdm_data_type:                      Point
    references:                         More information can be found at http...
    Metadata_Conventions:               Unidata Dataset Discovery v1.0
    date_created:                       2020-08-16T12:20:33.910735
    id:                                 CE02SHSM-SBD11-04-VELPTA000-recovered...
    requestUUID:                        06b0f3a7-e6c1-4491-b52d-48846c09fb14
    contributor_role:                   
    summary:                            Dataset Generated by Stream Engine fr...
    keywords_vocabulary:                
    institution:                        Ocean Observatories Initiative
    naming_authority:                   org.oceanobservatories
    feature_Type:                       point
    infoUrl:                            http://oceanobservatories.org/
    license:                            
    contributor_name:                   
    uuid:                               06b0f3a7-e6c1-4491-b52d-48846c09fb14
    creator_name:                       Ocean Observatories Initiative
    title:                              Data produced by Stream Engine versio...
    sensor:                             04-VELPTA000
    standard_name_vocabulary:           NetCDF Climate and Forecast (CF) Meta...
    acknowledgement:                    
    Conventions:                        CF-1.6
    project:                            Ocean Observatories Initiative
    source:                             CE02SHSM-SBD11-04-VELPTA000-recovered...
    publisher_url:                      http://oceanobservatories.org/
    creator_url:                        http://oceanobservatories.org/
    nodc_template_version:              NODC_NetCDF_TimeSeries_Orthogonal_Tem...
    subsite:                            CE02SHSM
    processing_level:                   L2
    history:                            2020-08-16T12:20:33.910616 generated ...
    Manufacturer:                       Nortek
    ModelNumber:                        Aquadopp
    SerialNumber:                       AQD 12742
    Description:                        Single Point Velocity Meter: VELPT Se...
    FirmwareVersion:                    Not specified.
    SoftwareVersion:                    Not specified.
    AssetUniqueID:                      CGINS-VELPTA-12742
    Notes:                              Not specified.
    Owner:                              Not specified.
    RemoteResources:                    []
    ShelfLifeExpirationDate:            Not specified.
    Mobile:                             False
    AssetManagementRecordLastModified:  2020-08-14T18:41:32.787000
    time_coverag

In [48]:
ds_merged['bin_depths_avg']

<xarray.DataArray 'bin_depths_avg' (bin_depths_avg: 37)>
array([ 1.893978,  9.147893, 10.147893, 11.147893, 12.147893, 13.147893,
       14.147893, 15.147893, 16.147893, 17.147893, 18.147893, 19.147893,
       20.147893, 21.147893, 22.147893, 23.147893, 24.147893, 25.147893,
       26.147893, 27.147893, 28.147893, 29.147893, 30.147893, 31.147893,
       32.147893, 33.147893, 34.147893, 74.602792, 70.602792, 66.602792,
       62.602792, 58.602792, 54.602792, 50.602792, 46.602792, 42.602792,
       38.602792])
Coordinates:
  * bin_depths_avg  (bin_depths_avg) float64 1.894 9.148 10.15 ... 42.6 38.6
Attributes:
    units:    m

In [50]:
ds_merged_sorted = ds_merged.sortby('bin_depths_avg')

In [51]:
plt.figure()
for i in range(len(ds_merged_sorted['bin_depths_avg'])):
    plt.plot([0,ds_merged_sorted['eastward_seawater_velocity'].mean(dim='time')[i]],
             [0,ds_merged_sorted['northward_seawater_velocity'].mean(dim='time')[i]]
             ,'-');
plt.axis('equal')
plt.xlabel('u [m/s]')
plt.ylabel('v [m/s]')
plt.title('mean current vectors')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'mean current vectors')

In [55]:
ds_merged_sorted_resampled = ds_merged_sorted.resample(time='15T').mean()

In [56]:
ds_merged_sorted_resampled.to_netcdf('nc_files/merged_velocity_orshelf.nc')